# Capítulo 5: Inferencia y grados de libertad

In [1]:
from pathlib import Path
DATA_PATH = Path("../data/AmesHousing_codificada.csv")  # relativo a book/notebooks/
assert DATA_PATH.is_file(), "No se encontró '../data/AmesHousing_codificada.csv'"
print("Usando CSV:", DATA_PATH.resolve())

Usando CSV: /workspaces/ames-housing-project/book/data/AmesHousing_codificada.csv


---

## **1️⃣ Definición de grados de libertad**

En un modelo lineal con $n$ observaciones y $k$ variables explicativas:

$$
Df_{Model} = k, \quad Df_{Residuals} = n - k - 1
$$

- $Df_{Model}$: número de parámetros estimados (sin contar el intercepto).  
- $Df_{Residuals}$: grados de libertad asociados a los errores o residuos.

---

## **2️⃣ Errores estándar y valores p**

El **error estándar** de cada coeficiente $\hat{\beta}_j$ mide la precisión de su estimación y se calcula como:

$$
SE(\hat{\beta}_j) = \sqrt{\sigma^2 (X^\top X)^{-1}_{jj}}
$$

donde:

- $\sigma^2 = \frac{SSE}{n - k - 1}$ es la varianza residual estimada.  
- $(X^\top X)^{-1}_{jj}$ es el elemento $j$-ésimo de la diagonal de la matriz inversa.

El **estadístico t** se define como:

$$
t_j = \frac{\hat{\beta}_j}{SE(\hat{\beta}_j)}
$$

y el **valor p** se obtiene comparando este estadístico con una distribución t de Student con $Df_{Residuals}$ grados de libertad.

---

## **3️⃣ Interpretación de la tabla de coeficientes**

Los resultados del modelo incluyen:

| Parámetro | Descripción |
|------------|--------------|
| **coef** | Estimación del parámetro $\hat{\beta}_j$ |
| **std err** | Error estándar del estimador |
| **t** | Estadístico de prueba |
| **P>|t|** | Valor p de la hipótesis $H_0: \beta_j = 0$ |
| **[0.025, 0.975]** | Intervalo de confianza al 95% |

---

## **4️⃣ Significancia y efecto práctico**

- Si $p < 0.05$: se **rechaza $H_0$** y la variable es **estadísticamente significativa**.  
- La **magnitud** del coeficiente indica su **efecto práctico**: cuánto cambia $y$ ante un cambio unitario en esa variable, manteniendo las demás constantes.

Por ejemplo:
- Un aumento de 1 punto en `OverallQual` puede incrementar el precio promedio en miles de dólares.
- `GrLivArea` tiene un efecto proporcional: cada metro adicional eleva el valor de la vivienda.

---

## **5️⃣ Resumen conceptual**

$$
\hat{\beta}_j \pm t_{\alpha/2, Df_{Residuals}} \times SE(\hat{\beta}_j)
$$

Este intervalo permite construir intervalos de confianza al 95% para cada parámetro, mostrando el rango plausible donde se encuentra el valor real de $\beta_j$.

In [2]:
import pandas as pd
import statsmodels.api as sm

# Usar el dataset limpio
df = pd.read_csv(DATA_PATH)
vars_modelo = ['Overall Qual','Gr Liv Area','Garage Cars','Garage Area','Total Bsmt SF','1st Flr SF','Year Built','Year Remod/Add','Full Bath','Garage Yr Blt','TotRms AbvGrd','Fireplaces','Mas Vnr Area','BsmtFin SF 1']

# Modelo OLS
X_sm = sm.add_constant(df[vars_modelo])
y = df["SalePrice"]
modelo = sm.OLS(y, X_sm).fit()

# Resumen de inferencia
tabla_inferencia = modelo.summary2().tables[1]
tabla_inferencia = tabla_inferencia.rename(columns={
    'Coef.': 'coef',
    'Std.Err.': 'std err',
    'P>|t|': 'p-value'
})

# Mostrar tabla
tabla_inferencia

,coef,std err,t,p-value,[0.025,0.975]
const,-1.321130e+06,64588.715168,-20.454506,1.107914e-86,-1.447777e+06,-1.194483e+06
Overall Qual,1.574852e+04,587.433210,26.809040,7.179209e-141,1.459667e+04,1.690037e+04
Gr Liv Area,4.856682e+01,2.341722,20.739785,6.732686e-89,4.397511e+01,5.315853e+01
Garage Cars,1.644127e+03,1558.156737,1.055175,2.914381e-01,-1.411147e+03,4.699402e+03
Garage Area,3.159232e+01,5.480444,5.764553,9.097526e-09,2.084612e+01,4.233851e+01
Total Bsmt SF,1.895102e+01,2.212417,8.565754,1.758644e-17,1.461286e+01,2.328919e+01
1st Flr SF,8.418598e+00,2.458300,3.424561,6.248038e-04,3.598300e+00,1.323890e+01
Year Built,1.955349e+02,30.369530,6.438522,1.418450e-10,1.359855e+02,2.550842e+02
Year Remod/Add,3.747123e+02,32.084298,11.678995,8.581986e-31,3.118006e+02,4.376241e+02
Full Bath,-4.656733e+03,1352.657854,-3.442654,5.846324e-04,-7.309060e+03,-2.004406e+03
